# Import libraries and read CSV file

Import necessary libraries and read CSV file

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

#Read Spider activity CSV file
filename = 'Metazygia wittfeldae Monitor 1 activity_LD_binary'
df = pd.read_csv(filename + '.csv', index_col = 0)
df1 = df.copy()
display(df1)

,Date_Time,Date,Time,lights,s1,s2,s3,s4,s5,s6,...,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32
0,2017-06-11 00:00:00,11-Jun-17,0:00:00,0,1,1,0,0,0,0,...,1,1,0,0,0,0,0,1,1,0
1,2017-06-11 00:01:00,11-Jun-17,0:01:00,0,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
2,2017-06-11 00:02:00,11-Jun-17,0:02:00,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,2017-06-11 00:03:00,11-Jun-17,0:03:00,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,2017-06-11 00:04:00,11-Jun-17,0:04:00,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,2017-06-18 23:55:00,18-Jun-17,23:55:00,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
11516,2017-06-18 23:56:00,18-Jun-17,23:56:00,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,1
11517,2017-06-18 23:57:00,18-Jun-17,23:57:00,0,0,1,0,0,0,1,...,1,1,0,0,0,0,0,0,0,1
11518,2017-06-18 23:58:00,18-Jun-17,23:58:00,0,0,1,0,0,0,1,...,1,1,0,0,0,0,0,0,0,0


# Switch Function

Identifies where spider started and stopped moving and checks corner cases (start of dataframe row and end of dataframe row). 

In [50]:
def adjust_start_and_end(df_switch, x):
    df_switch['switch'] = df_switch[x].diff()
    
    if df_switch.at[0, x] == 1:
            df_switch.at[0, 'switch'] = 1
    
    if df_switch.at[df_switch.index[-1], x] == 1 and df_switch.at[df_switch.index[-2], x] == 1:
            df_switch.at[df_switch.index[-1], 'switch'] = -1
    
    if df_switch.at[df_switch.index[-1], x] == 1 and df_switch.at[df_switch.index[-2], x] == 0:
            df_switch.at[df_switch.index[-1], 'switch'] = 0

# Duration Function

Creates two lists; runStart is a list with the indices of when the spider started moving, runEnd is a list with the indices of when the spider stopped moving. Run duration is found by subtracting these two lists.

In [56]:
def find_duration(df_switch, x):
    runStart = df_switch.index[df_switch['switch'] == 1].tolist()
    runEnd = df_switch.index[df_switch['switch'] == -1].tolist()
    runDuration = np.subtract(runEnd, runStart)
    
    return runDuration

# New Dataframe

Making a new dataframe with run durations for all spiders. If a spider has shorter run duration counts than another spider, the empty rows are filled with NaN. Dataframe is then made into a CSV file.

In [58]:
def allSpiderRuns(df1):
    newdf = pd.DataFrame()
    df_switch = df1.copy()
    
    for x in df1.loc[:, df1.columns.str.startswith('s')]:
        
        adjust_start_and_end(df_switch, x)
        
        runDuration = find_duration(df_switch, x)
    
        s = pd.Series(runDuration)
        #appends columns and fills columns outside the intersection with NaN values
        newdf = pd.concat([newdf, s.rename(x)], axis=1) 
        #converts all cells to float
        newdf[x] = newdf[x].astype(float)
    
    display(newdf)
    newdf.to_csv(filename + "_run_duration.csv")    

allSpiderRuns(df1)

,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,...,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32
0,1.0,23.0,6.0,6.0,18.0,1.0,2.0,11.0,1.0,7.0,...,1.0,2.0,3.0,2.0,2.0,3.0,9.0,1.0,1.0,6.0
1,1.0,7.0,6.0,2.0,16.0,12.0,10.0,6.0,1.0,1.0,...,12.0,1.0,1.0,2.0,1.0,1.0,3.0,21.0,1.0,6.0
2,1.0,4.0,8.0,1.0,3.0,2.0,4.0,7.0,1.0,13.0,...,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,5.0
3,5.0,5.0,1.0,1.0,5.0,3.0,1.0,3.0,5.0,5.0,...,2.0,1.0,10.0,3.0,5.0,1.0,10.0,1.0,4.0,3.0
4,2.0,5.0,1.0,19.0,7.0,2.0,5.0,5.0,1.0,1.0,...,4.0,2.0,2.0,1.0,3.0,1.0,8.0,3.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
805,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
806,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
